## load data and main preparing

In [1]:
import pandas as pd
data=pd.read_csv("reuters_data.csv")
data.head(7)

,REUTERS TOPICS,TOPICS,TITLE,BODY
0,YES,cocoa,BAHIA COCOA REVIEW,Showers continued throughout the week in the B...
1,YES,grain,NATIONAL AVERAGE PRICES FOR FARMER-OWNED RESERVE,The US Agriculture Department reported the far...
2,YES,veg-oil,ARGENTINE 1986/87 GRAIN/OILSEED REGISTRATIONS,Argentine grain board figures show crop regist...
3,YES,none,USX <X> DEBT DOWGRADED BY MOODY'S,Moodys Investors Service Inc said it lowered t...
4,YES,earn,CHAMPION PRODUCTS <CH> APPROVES STOCK SPLIT,Champion Products Inc said its board of direct...
5,YES,acq,COMPUTER TERMINAL SYSTEMS <CPML> COMPLETES SALE,Computer Terminal Systems Inc said it has comp...
6,YES,earn,COBANCO INC <CBCO> YEAR NET,Shr 34 cts vs 119 dlrs Net 807000 vs 2858000 A...


In [2]:
data=data.dropna()
data=data.drop_duplicates()
len(data)

12924

In [3]:
print(len(data[data['REUTERS TOPICS']!='YES']))
#only 5 rows ,so may be incorrect data so remove the rows
data = data[data['REUTERS TOPICS'] == 'YES']
len(data)

5


12919

In [4]:
#all REUTERS TOPICS is YES so drop unnecessary column
data = data.drop('REUTERS TOPICS', axis=1)


In [5]:
#test is okay removed
data[data["TOPICS"]=='none']

,TOPICS,TITLE,BODY
3,none,USX <X> DEBT DOWGRADED BY MOODY'S,Moodys Investors Service Inc said it lowered t...
12,none,HUGHES CAPITAL UNIT SIGNS PACT WITH BEAR STEARNS,HughesConserdyne Corp a unit of Hughes Capital...
16,none,BANKERS REPORT BREAKTHROUGH ON VENEZUELAN DEBT,Venezuela and its bank advisory committee have...
20,none,FIRST UNION <FUNC> FILES 100 MLN DLR NOTES ISSUE,First Union Corp said it has filed with the Se...
39,none,ARGENTINA COULD SUSPEND DEBT PAYMENTS - DEPUTY,Argentina could suspend payments on its foreig...
...,...,...,...
12554,none,WORLD BANK EXTENDS 241 MLN DLRS IN AFRICAN LOANS,The World Bank said it has extended loans tota...
12593,none,IMF APPROVES 48 MLN DLR LOAN TO ASSIST ECUADOR,The International Monetary Fund said it approv...
12605,none,EXPORT CREDIT BANK FOUNDED BY CARIBBEAN NATIONS,Finance ministers from the 13member Caribbean ...
12638,none,AQUINO SAYS GROWTH HAS PRIORITY OVER DEBT PAYM...,President Corazon Aquino said economic growth ...


In [6]:
data_topicNone=data[data["TOPICS"]=='none']#will be used later to be predicated
data=data[data["TOPICS"]!='none']

In [7]:
len(data[data["TOPICS"]=="none"])
#missed data, will be filled later by the clustering algorithm

0

In [8]:
from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
data['TOPICS CODE'] = label_encoder.fit_transform(data['TOPICS'])
data.head(7)

,TOPICS,TITLE,BODY,TOPICS CODE
0,cocoa,BAHIA COCOA REVIEW,Showers continued throughout the week in the B...,6
1,grain,NATIONAL AVERAGE PRICES FOR FARMER-OWNED RESERVE,The US Agriculture Department reported the far...,23
2,veg-oil,ARGENTINE 1986/87 GRAIN/OILSEED REGISTRATIONS,Argentine grain board figures show crop regist...,75
4,earn,CHAMPION PRODUCTS <CH> APPROVES STOCK SPLIT,Champion Products Inc said its board of direct...,16
5,acq,COMPUTER TERMINAL SYSTEMS <CPML> COMPLETES SALE,Computer Terminal Systems Inc said it has comp...,0
6,earn,COBANCO INC <CBCO> YEAR NET,Shr 34 cts vs 119 dlrs Net 807000 vs 2858000 A...,16
7,earn,OHIO MATTRESS <OMT> MAY HAVE LOWER 1ST QTR NET,Ohio Mattress Co said its first quarter ending...,16


In [24]:
#get distnict topics
distinictTypes=set(data["TOPICS"].values)
print(len(distinictTypes))

81


In [ ]:
# download nltk
import nltk
nltk.download('all')

## sample to easy computation , more prepare words

In [16]:
#classify where X is TITLE,BODY ,y is TOPIC
import numpy as np
#take random simple to easy computation
sample = data#data.sample(n=500, replace=True,random_state=123) 
sample = sample.reset_index(drop=True)#to make index from 0
sample

,TOPICS,TITLE,BODY,TOPICS CODE
0,cocoa,BAHIA COCOA REVIEW,Showers continued throughout the week in the B...,6
1,grain,NATIONAL AVERAGE PRICES FOR FARMER-OWNED RESERVE,The US Agriculture Department reported the far...,23
2,veg-oil,ARGENTINE 1986/87 GRAIN/OILSEED REGISTRATIONS,Argentine grain board figures show crop regist...,75
3,earn,CHAMPION PRODUCTS <CH> APPROVES STOCK SPLIT,Champion Products Inc said its board of direct...,16
4,acq,COMPUTER TERMINAL SYSTEMS <CPML> COMPLETES SALE,Computer Terminal Systems Inc said it has comp...,0
...,...,...,...,...
10863,earn,EASTERN UTILITIES ASSOCIATES 3RD QTR NET,Shr 86 cts vs 74 cts Net 111 mln vs 86 mln Rev...,16
10864,trade,"EC, U.S. PLAN HIGH-LEVEL TRADE TALKS",The European Community is willing to offer lim...,74
10865,crude,BRITAIN BACKS U.S. STRIKE ON IRAN OIL PLATFORM,British Foreign Secretary Sir Geoffrey Howe ba...,14
10866,acq,SIMON AND SCHUSTER TO ACQUIRE WOODHEAD-FAULKNE...,SIMON AND SCHUSTER TO ACQUIRE WOODHEADFAULKNER...,0


In [14]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
def lemmatize(column):
    text = list(column)
    lemmatizer = WordNetLemmatizer()
    corpus = []
    stemmer = PorterStemmer()
    for i in range(len(text)):
        r = re.sub('[^a-zA-Z]', ' ', text[i])
        r = r.lower()
        r = r.split()
        r = [word for word in r if word not in stopwords.words('english')]
        r = [lemmatizer.lemmatize(word) for word in r]
        r = [stemmer.stem(word) for word in r]
        r = ' '.join(r)
        corpus.append(r)
    return corpus    

In [17]:
sample['TITLE']=lemmatize(sample['TITLE'])
sample['BODY']=lemmatize(sample['BODY'])



In [ ]:
sample.head()


## tf-idf of clomn "body"

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf = TfidfVectorizer()
tf_idf_matrix = tf_idf.fit_transform(sample['BODY'].values)
tf_idf_matrix_arr = tf_idf_matrix.toarray()
tf_idf_matrix_arr = pd.DataFrame(tf_idf_matrix_arr, columns=tf_idf.get_feature_names_out())
tf_idf_matrix_arr

In [ ]:
# Calculate the average TF-IDF score for each term
average_scores = tf_idf_matrix_arr.mean()
average_scores

In [ ]:
# Find the top 5 terms with the highest average TF-IDF scores
top_terms = average_scores.nlargest(5)

print("Top 5 Terms with Highest Average TF-IDF Scores:")
for term, score in top_terms.iteritems():
    print(term, ":", score)

## visualize

### barplot of word count

In [ ]:
import matplotlib.pyplot as plt
topic_counts = sample['BODY'].value_counts()
# Plot the bar chart
plt.figure(figsize=(10, 6))
plt.bar(topic_counts.index, topic_counts.values)
plt.title('Topic Distribution')
plt.xlabel('Topic')
plt.ylabel('Count')
plt.xticks(rotation=90)
plt.show()

### wordcloud

In [ ]:
!pip install WordCloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt


wordcloud = WordCloud(width=800, height=400, background_color='white').generate(sample['BODY'].astype(str).str.cat(sep=' '))

# Display the generated image:
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()


## analysis

### NN classify is returs or not

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_, y_train, y_ = train_test_split(sample[['TITLE','BODY']], sample['TOPICS CODE'], test_size=0.3, random_state=123)
X_cv, X_test, y_cv, y_test = train_test_split(X_, y_, test_size=0.5, random_state=123)
del(X_)
del(y_)
X_train.shape

(7607, 2)

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import hstack

def vectorize(df,df_cv,df_tst):
    vectorizer1 = CountVectorizer()
    df_Title = vectorizer1.fit_transform(df['TITLE'])
    df_cv_title=vectorizer1.transform(df_cv['TITLE'])
    df_tst_title=vectorizer1.transform(df_tst['TITLE'])

    
    vectorizer2 = CountVectorizer()
    df_body = vectorizer2.fit_transform(df['BODY'])
    df_cv_body=vectorizer2.transform(df_cv['BODY'])
    df_tst_body=vectorizer2.transform(df_tst['BODY'])

    df_combined = hstack([df_Title,df_body])
    df_cv_combined = hstack([df_cv_title,df_cv_body])
    df_tst_combined = hstack([df_tst_title,df_tst_body])

    return (df_combined,df_cv_combined,df_tst_combined)

In [20]:
vc_train,vc_cv,vc_test=vectorize(X_train,X_cv,X_test)


In [21]:
vc_cv.shape

(1630, 25464)

In [22]:
y_cv.shape

(1630,)

In [25]:

from tensorflow import keras
model = keras.Sequential([
    keras.layers.Dense(100, activation='relu'),  
    keras.layers.Dense(100, activation='relu'),  
    keras.layers.Dense(100, activation='relu'),  
    keras.layers.Dense(len(distinictTypes), activation='softmax')  
])

model.compile(optimizer='adam', loss='SparseCategoricalCrossentropy', metrics=['accuracy'])
model.fit(np.array(vc_train.toarray()), y_train, epochs=10,batch_size=32)
loss, accuracy = model.evaluate(np.array(vc_cv.toarray()), y_cv)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy*100:.2f}%")


Epoch 1/10
238/238 [==============================] - 12s 45ms/step - loss: 1.3096 - accuracy: 0.7320
Epoch 2/10
238/238 [==============================] - 13s 53ms/step - loss: 0.3137 - accuracy: 0.9314
Epoch 3/10
238/238 [==============================] - 12s 49ms/step - loss: 0.1138 - accuracy: 0.9753
Epoch 4/10
238/238 [==============================] - 11s 47ms/step - loss: 0.0639 - accuracy: 0.9845
Epoch 5/10
238/238 [==============================] - 11s 48ms/step - loss: 0.0369 - accuracy: 0.9896
Epoch 6/10
238/238 [==============================] - 11s 46ms/step - loss: 0.0309 - accuracy: 0.9913
Epoch 7/10
238/238 [==============================] - 11s 45ms/step - loss: 0.0206 - accuracy: 0.9934
Epoch 8/10
238/238 [==============================] - 10s 43ms/step - loss: 0.0195 - accuracy: 0.9933
Epoch 9/10
238/238 [==============================] - 10s 44ms/step - loss: 0.0187 - accuracy: 0.9941
Epoch 10/10
51/51 [==============================] - 1s 9ms/step - loss: 0.6723 - 

In [26]:
loss, accuracy = model.evaluate(np.array(vc_test.toarray()), y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy*100:.2f}%")

51/51 [==============================] - 0s 8ms/step - loss: 0.6876 - accuracy: 0.8761
Test Loss: 0.6876, Test Accuracy: 87.61%


In [30]:
#save model
model.save('NN_model_all_data_without_none.h5')


In [28]:
## predict for none topic-given data
vec = CountVectorizer()#this to get the countvectorizer used in training model
vec_title = vec.fit_transform(X_train['TITLE'])
vec2 = CountVectorizer()
vec_BODY = vec2.fit_transform(X_train['BODY'])

data_topicNone_title=vec.transform(data_topicNone['TITLE'])
data_topicNone_body=vec2.transform(data_topicNone['BODY'])

data_topicNone_encoded = hstack([data_topicNone_title,data_topicNone_body])
data_topicNone_encoded

<2051x25464 sparse matrix of type '<class 'numpy.int64'>'
	with 71721 stored elements in Compressed Sparse Row format>

In [34]:
predictions = model.predict(data_topicNone_encoded)
predicted_class = np.argmax(predictions, axis=-1)
data_topicNone['TOPICS']= label_encoder.inverse_transform(predicted_class)
data_topicNone

65/65 [==============================] - 0s 2ms/step


,TOPICS,TITLE,BODY,TOPICS CODE
3,earn,USX <X> DEBT DOWGRADED BY MOODY'S,Moodys Investors Service Inc said it lowered t...,16
12,acq,HUGHES CAPITAL UNIT SIGNS PACT WITH BEAR STEARNS,HughesConserdyne Corp a unit of Hughes Capital...,0
16,interest,BANKERS REPORT BREAKTHROUGH ON VENEZUELAN DEBT,Venezuela and its bank advisory committee have...,31
20,acq,FIRST UNION <FUNC> FILES 100 MLN DLR NOTES ISSUE,First Union Corp said it has filed with the Se...,0
39,earn,ARGENTINA COULD SUSPEND DEBT PAYMENTS - DEPUTY,Argentina could suspend payments on its foreig...,16
...,...,...,...,...
12554,earn,WORLD BANK EXTENDS 241 MLN DLRS IN AFRICAN LOANS,The World Bank said it has extended loans tota...,16
12593,money-fx,IMF APPROVES 48 MLN DLR LOAN TO ASSIST ECUADOR,The International Monetary Fund said it approv...,43
12605,trade,EXPORT CREDIT BANK FOUNDED BY CARIBBEAN NATIONS,Finance ministers from the 13member Caribbean ...,74
12638,interest,AQUINO SAYS GROWTH HAS PRIORITY OVER DEBT PAYM...,President Corazon Aquino said economic growth ...,31


### Markov to generate text same way as data

In [ ]:
!pip install markovify

In [11]:
######

def load(data):    
    corpus = ""
    for i in range(len(data)):      
            corpus += " "+data['BODY'][i]
    return corpus


In [57]:
sample2 = data.sample(n=400, replace=True)
sample2 = sample2.reset_index(drop=True)#to make index from 0

import markovify
import sys
text = load(sample2)
# Train model
text_model = markovify.Text(text)

# Generate sentences
print()
for i in range(5):
    print(text_model.make_sentence(max_attempts=100))
    print()



None

None

REVLON BUYS GERMAINE MONTEIL COSMETICS FROM BEECHAM GROUP FOR UNDISCLOSED TERMS Pakistan bought 6000 tonnes for first three months after the incident Lloyds Shipping Intelligence reported The 315695dwt Arabian Sea had set aside for an incentive stock option plan Reuter

None

None



## similarity

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

vectorizer = CountVectorizer()
sam_body = vectorizer.fit_transform(sample['BODY'])

# Compute the cosine similarity matrix
cosine_sim_matrix = cosine_similarity(sam_body,sam_body)

# Display the similarity matrix
similarity_df = pd.DataFrame(cosine_sim_matrix, columns=sample.index, index=sample.index)



In [ ]:
similarity_df

In [ ]:
# save to csv 
try:
    similarity_df.to_csv('similarity_matrix.csv')
    print("successfully created")
except Exception as e:
    print(f"Error saving the file: {e}")